_Author: Stephen Carr_

In [1]:
using DMRJtensor
include("su2_tools.jl");


In [32]:
# setup system with isotropic J
spinmag = 1.5;
Sx,Sy,Sz,Sp,Sm,O,Id = spinOps(spinmag)
Ns = 20 # number of sites
nSpin = size(Sx,1)
max_p = nSpin-1


#J_arr = 2.0*(rand(max_p,1) .- 0.5) # random values between [-1,1]
J_arr = rand(max_p,1) # random values between [0,1]

J_arr = J_arr ./ maximum(broadcast(abs,J_arr))
display(J_arr)
H_onesite, H_op_vec = H_SU2(spinmag, J_arr) # make the onsite term

H_mpo = makeMPO(H_onesite,size(Id,1),Ns); # make the MPO!
psi0 = makePsi0(spinmag,Ns);
psi = copy(psi0)
println(psi[1])

3×1 Array{Float64,2}:
 0.18961747332757337
 1.0                
 0.06561242631240016

printing regular tensor of type: tens{Complex{Float64}}
size = (1, 4, 1)
T = Complex{Float64}[0.27738283657590235 + 0.3624806138193597im, 0.2876832295911138 + 0.13148332679484054im, 0.9759426147167469 + 0.38246237190211696im, 0.42843423217216925 + 0.8319175827530845im]



In [33]:
nOps = length(H_op_vec)
MPO_op_vec = []
for idx = 1:nOps
    MPO_here = makeMPO(H_op_vec[idx],size(Id,1),Ns);
    push!(MPO_op_vec, MPO_here)
end


In [34]:
# DMRG sweeps, with refinement 

nsweeps_m10 = 20
nsweeps_m40 = 10
nsweeps_m80 = 5

for i in 1:nsweeps_m10
    @time dmrg(psi, H_mpo, maxm = 10, cutoff = 1E-4)
end
println("coarse run completed")
variance = expect(psi,H_mpo,H_mpo)- (expect(psi, H_mpo))^2
println("variance is, ",variance)

for i in 1:nsweeps_m40
    @time dmrg(psi, H_mpo, maxm = 40, cutoff = 1E-8)
end
println("intermediate run completed")
#for i in 1:nsweeps_m80
#    @time dmrg(psi, H_mpo, maxm = 80, cutoff = 1E-15)
#end
#println("fine run completed")
variance = expect(psi,H_mpo,H_mpo)- (expect(psi, H_mpo))^2
println("variance is, ",variance)

  0.114850 seconds (60.79 k allocations: 46.713 MiB, 1.79% gc time)
  0.216313 seconds (90.91 k allocations: 102.375 MiB, 3.82% gc time)
  0.191073 seconds (91.50 k allocations: 103.646 MiB, 2.54% gc time)
  0.167483 seconds (93.31 k allocations: 106.965 MiB, 2.91% gc time)
  0.162116 seconds (93.44 k allocations: 107.156 MiB, 2.97% gc time)
  0.167147 seconds (93.76 k allocations: 107.819 MiB, 4.06% gc time)
  0.171561 seconds (94.02 k allocations: 108.315 MiB, 2.82% gc time)
  0.161233 seconds (94.25 k allocations: 108.808 MiB, 3.01% gc time)
  0.186441 seconds (94.34 k allocations: 108.954 MiB, 3.66% gc time)
  0.192767 seconds (94.28 k allocations: 108.797 MiB, 2.53% gc time)
  0.193727 seconds (94.28 k allocations: 108.797 MiB, 3.74% gc time)
  0.178864 seconds (94.25 k allocations: 108.796 MiB, 2.74% gc time)
  0.167287 seconds (94.25 k allocations: 108.796 MiB, 3.17% gc time)
  0.197934 seconds (94.25 k allocations: 108.796 MiB, 3.90% gc time)
  0.184125 seconds (94.25 k allocat

In [35]:
# evaluate Correlation matrix

corMat = zeros(ComplexF64,nOps,nOps)

op_expect = zeros(ComplexF64,nOps)

for idx = 1:nOps
    op_expect[idx] = expect(psi,MPO_op_vec[idx])
end

for idx1 = 1:nOps
    for idx2 = 1:nOps
        val_h = 0.5*expect(psi,MPO_op_vec[idx1],MPO_op_vec[idx2])
        val_h += 0.5*expect(psi,MPO_op_vec[idx2],MPO_op_vec[idx1])
        val_h += -(op_expect[idx1]*op_expect[idx2])
        corMat[idx1,idx2] = val_h
    end
end
broadcast(abs,corMat)

3×3 Array{Float64,2}:
  40.988   19.1427   174.372 
  19.1427   9.68336   91.3228
 174.372   91.3228   887.171 

In [36]:
# check the symmetry of M
maximum(broadcast(abs,corMat - corMat'))

9.187072170939457e-13

In [43]:
using LinearAlgebra
using Printf

#keep_idx = 1:nOps # all operators
keep_idx = [1,3]#1:(nOps-1) # drop last operator
corMat_reduced = corMat[keep_idx,keep_idx]

vals,vecs = LinearAlgebra.eigen(corMat_reduced)
min_val, tar_idx = findmin(broadcast(abs,vals))
#min_val, tar_idx = findmin(broadcast(real,vals))


op_estimate = broadcast(abs,vecs[:,tar_idx])/maximum(broadcast(abs,vecs[:,tar_idx]))

display("J_arr:")
display(J_arr)

@printf("eigenvalues = ")
for val in vals
    @printf("%.5f ",real(val))
end
@printf("\n")

@printf("min_eig = %E \n",min_val)

@printf("operators = ")
for op in op_estimate
    @printf("%.5f ",op)
end
@printf("\n")


"J_arr:"

3×1 Array{Float64,2}:
 0.18961747332757337
 1.0                
 0.06561242631240016

eigenvalues = 6.46412 921.69503 
min_eig = 6.464125E+00 
operators = 1.00000 0.19799 


In [38]:
# Evaluate and plot the site-dependent magnetization
S_vals = zeros(ComplexF64,Ns,3)
S_ops = [Sx, Sy, Sz]

S0 = [Id O 
      O Id]
S0_mpo = Array{Array{ComplexF64,2},1}(undef,Ns)

for site = 1:Ns
   S0_mpo[site] = S0
end

for site = 1:Ns
    for op_idx = 1:3
        Sop_here = copy(S0_mpo)
        Sop_here[site] = [Id            O
                          S_ops[op_idx] Id]
        mpo_h = makeMPO(Sop_here,size(Id,1),Ns)
        S_vals[site,op_idx] = expect(psi,mpo_h)
    end
end

using Plots
plot(broadcast(real,S_vals[:,1]),label="Sx")
plot!(broadcast(real,S_vals[:,2]),label="Sy")
plot!(broadcast(real,S_vals[:,3]),label="Sz")

InterruptException: [91mInterruptException:[39m

In [172]:
tar_vec = H_op_vec[12]; # 12 corresponds to Sz^2

display("Op mid left:")
display(tar_vec[4:6,1:3])

display("Op bot mid:")
display(tar_vec[7:9,4:6])

display("Id top left:")
display(tar_vec[1:3,1:3])

display("Id bot right:")
display(tar_vec[7:9,7:9])

display("Zero bot left:")
display(tar_vec[7:9,1:3])

display("rest:")
display(tar_vec[1:6,4:9])



"Op mid left:"

3×3 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im

"Op bot mid:"

3×3 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im

"Id top left:"

...

3×3 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im

"Zero bot left:"

3×3 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im

"rest:"

6×6 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [76]:
# check the term in the Hamiltonian (top left part is multiplied by respective J)
tar_op = 4;
guess_op = Sx*Sx;

nH_rows = size(H_onesite)[1]
lrs = nH_rows-nSpin+1; # Last Row Starting index
s = (tar_op)*nSpin;

guess_op_r = guess_op
guess_op_l = guess_op
op_actual_r = H_onesite[(1+s):(3+s),        1:3]
op_actual_l = H_onesite[lrs:(lrs+2),(1+s):(3+s)]
op_r_err = J_arr[1]*guess_op_r - op_actual_r
op_l_err =          guess_op_l - op_actual_l
display(maximum(broadcast(abs,op_r_err[:])))
display(maximum(broadcast(abs,op_l_err[:])))
display(guess_op_l)
display(op_actual_l)


0.6068704789525406

1.0000000000000002

3×3 Array{Float64,2}:
 0.5  0.0  0.5
 0.0  1.0  0.0
 0.5  0.0  0.5

3×3 Array{Complex{Float64},2}:
      0.0+0.0im  0.707107+0.0im       0.0+0.0im
 0.707107+0.0im       0.0+0.0im  0.707107+0.0im
      0.0+0.0im  0.707107+0.0im       0.0+0.0im

3×3 Array{Float64,2}:
 0.5  0.0  0.5
 0.0  1.0  0.0
 0.5  0.0  0.5